In [1]:
import cv2
import numpy as np
import cvzone
from cvzone.SelfiSegmentationModule import SelfiSegmentation
from skimage import metrics
import torch
import open_clip
import cv2
from sentence_transformers import util
from PIL import Image



/Users/caroline.hughes/anaconda3/envs/py38_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Load Images

In [2]:
real_gucci_1_logo = ("/users/caroline.hughes/Documents/github datasets/real gucci logo.png")
fake_gucci_1_logo = ("/users/caroline.hughes/Documents/github datasets/fake gucci logo.png")
real_gucci_2_logo = ("/users/caroline.hughes/Documents/github datasets/real gucci logo 2.png")

real_gucci_1_hardware = ("/users/caroline.hughes/Documents/github datasets/real gucci hardware.png")
fake_gucci_1_hardware = ("/users/caroline.hughes/Documents/github datasets/fake gucci hardware.png")
real_gucci_2_hardware = ("/users/caroline.hughes/Documents/github datasets/real gucci hardware 2.png")


real_gucci_1_pattern = ("/users/caroline.hughes/Documents/github datasets/real gucci pattern.png")
fake_gucci_1_pattern = ("/users/caroline.hughes/Documents/github datasets/fake gucci pattern.png")
real_gucci_2_pattern = ("/users/caroline.hughes/Documents/github datasets/real gucci pattern 2.png")

real_gucci_1_label_bag = ("/users/caroline.hughes/Documents/github datasets/real gucci label bag.png")
fake_gucci_1_label_bag = ("/users/caroline.hughes/Documents/github datasets/fake gucci label bag.png")
real_gucci_2_label_bag = ("/users/caroline.hughes/Documents/github datasets/real gucci label bag 2.png")

real_gucci_1_label_hoodie = ("/users/caroline.hughes/Documents/github datasets/real gucci label hoodie.png")
fake_gucci_1_label_hoodie = ("/users/caroline.hughes/Documents/github datasets/fake gucci label hoodie.png")
real_gucci_2_label_hoodie = ("/users/caroline.hughes/Documents/github datasets/real gucci label hoodie 2.png")


### Real VS Fake Gucci - Histogram-Based Approach

In [3]:
# Load images
image1 = cv2.imread(real_gucci_1_label_hoodie)
image2 = cv2.imread(fake_gucci_1_label_hoodie)
hist_img1 = cv2.calcHist([image1], [0, 1, 2], None, [256, 256, 256], [0, 256, 0, 256, 0, 256])
hist_img1[255, 255, 255] = 0 #ignore all white pixels
cv2.normalize(hist_img1, hist_img1, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)
hist_img2 = cv2.calcHist([image2], [0, 1, 2], None, [256, 256, 256], [0, 256, 0, 256, 0, 256])
hist_img2[255, 255, 255] = 0  #ignore all white pixels
cv2.normalize(hist_img2, hist_img2, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)
# Find the metric value
metric_val = cv2.compareHist(hist_img1, hist_img2, cv2.HISTCMP_CORREL)
print(f"Similarity Score: ", round(metric_val, 2))


Similarity Score:  -0.0


### Real vs Real Gucci - Histogram-Based Approach

In [4]:
# Load images
image1 = cv2.imread(real_gucci_1_label_hoodie)
image2 = cv2.imread(real_gucci_2_label_hoodie)
hist_img1 = cv2.calcHist([image1], [0, 1, 2], None, [256, 256, 256], [0, 256, 0, 256, 0, 256])
hist_img1[255, 255, 255] = 0 #ignore all white pixels
cv2.normalize(hist_img1, hist_img1, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)
hist_img2 = cv2.calcHist([image2], [0, 1, 2], None, [256, 256, 256], [0, 256, 0, 256, 0, 256])
hist_img2[255, 255, 255] = 0  #ignore all white pixels
cv2.normalize(hist_img2, hist_img2, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)
# Find the metric value
metric_val = cv2.compareHist(hist_img1, hist_img2, cv2.HISTCMP_CORREL)
print(f"Similarity Score: ", round(metric_val, 2))


Similarity Score:  -0.01


### Real vs Fake Gucci - SSIM Approach

In [5]:

# Load images
image1 = cv2.imread(real_gucci_1_label_hoodie)
image2 = cv2.imread(fake_gucci_1_label_hoodie)
image2 = cv2.resize(image2, (image1.shape[1], image1.shape[0]), interpolation = cv2.INTER_AREA)
print(image1.shape, image2.shape)
# Convert images to grayscale
image1_gray = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
image2_gray = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)
# Calculate SSIM
ssim_score = metrics.structural_similarity(image1_gray, image2_gray, full=True)
print(f"SSIM Score: ", round(ssim_score[0], 2))


(354, 351, 3) (354, 351, 3)
SSIM Score:  0.31


### Real vs Real Gucci - SSIM Approach

In [6]:

# Load images
image1 = cv2.imread(real_gucci_1_label_hoodie)
image2 = cv2.imread(real_gucci_2_label_hoodie)
image2 = cv2.resize(image2, (image1.shape[1], image1.shape[0]), interpolation = cv2.INTER_AREA)
print(image1.shape, image2.shape)
# Convert images to grayscale
image1_gray = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
image2_gray = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)
# Calculate SSIM
ssim_score = metrics.structural_similarity(image1_gray, image2_gray, full=True)
print(f"SSIM Score: ", round(ssim_score[0], 2))


(354, 351, 3) (354, 351, 3)
SSIM Score:  0.15


### Real vs Fake Gucci- Neural Network Approach

In [7]:

device = "cuda" if torch.cuda.is_available() else "cpu"
model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-16-plus-240', pretrained="laion400m_e32")
model.to(device)
def imageEncoder(img):
    img1 = Image.fromarray(img).convert('RGB')
    img1 = preprocess(img1).unsqueeze(0).to(device)
    img1 = model.encode_image(img1)
    return img1
def generateScore(image1, image2):
    test_img = cv2.imread(real_gucci_1_label_hoodie, cv2.IMREAD_UNCHANGED)
    data_img = cv2.imread(fake_gucci_1_label_hoodie, cv2.IMREAD_UNCHANGED)
    img1 = imageEncoder(test_img)
    img2 = imageEncoder(data_img)
    cos_scores = util.pytorch_cos_sim(img1, img2)
    score = round(float(cos_scores[0][0])*100, 2)
    return score
print(f"similarity Score: ", round(generateScore(image1, image2), 2))


similarity Score:  93.01


### Real vs Real Gucci- Neural Network Approach

In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-16-plus-240', pretrained="laion400m_e32")
model.to(device)
def imageEncoder(img):
    img1 = Image.fromarray(img).convert('RGB')
    img1 = preprocess(img1).unsqueeze(0).to(device)
    img1 = model.encode_image(img1)
    return img1
def generateScore(image1, image2):
    test_img = cv2.imread(real_gucci_1_label_hoodie, cv2.IMREAD_UNCHANGED)
    data_img = cv2.imread(real_gucci_2_label_hoodie, cv2.IMREAD_UNCHANGED)
    img1 = imageEncoder(test_img)
    img2 = imageEncoder(data_img)
    cos_scores = util.pytorch_cos_sim(img1, img2)
    score = round(float(cos_scores[0][0])*100, 2)
    return score
print(f"similarity Score: ", round(generateScore(image1, image2), 2))

similarity Score:  90.16


### Attempt at removing background noise

In [9]:
src = cv2.imread(real_gucci_1_logo, 1)
tmp = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
_,alpha = cv2.threshold(tmp,0,255,cv2.THRESH_BINARY)
b, g, r = cv2.split(src)
rgba = [b,g,r, alpha]
dst = cv2.merge(rgba,4)

cv2.imwrite("/users/caroline.hughes/Documents/github datasets/real gucci logo 1 edit.png", dst)


True